In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
import tensorflow as tf
from tensorflow import keras

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adadelta

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
from tqdm import tqdm_notebook as tqdm
from utils.dataloader import DataLoader

In [6]:
def data_gen(data_loader: DataLoader, size: int = 32, loops: int = -1, device=None):
    for x, xi, y, yi in data_loader.gen_batch(size, loops):
        x = torch.from_numpy(x).to(torch.long).to(device)
        xi = torch.from_numpy(xi).to(torch.long).to(device)
        y = torch.from_numpy(y).to(torch.long).to(device)
        yi = torch.from_numpy(yi).to(torch.long).to(device)
        yield x, xi, y, yi

In [7]:
data_loader = DataLoader()
data_loader

data:./data/zh.tsv
            series size:154988
            input data max length:51 words:1153
            target data max length:51 words:4461
            

In [10]:
class Tnet(nn.Module):
    def __init__(self,chars_count,target_count):
        super().__init__()
        self.embedding = nn.Embedding(chars_count,512)
        self.linear_rate = nn.Sequential(
            nn.Conv1d(512,512,kernel_size=1),
            nn.ReLU(),
        )
        self.r1 = nn.GRU(512,512,num_layers=2,batch_first=True,bidirectional=True)
        self.decoder = nn.Sequential(
            nn.Conv1d(1024,target_count,kernel_size=1),
            nn.ReLU(),
            nn.LogSoftmax(dim=1),
        )
        
        map(nn.init.xavier_normal_,self.parameters())

    def forward(self,x:torch.Tensor):
        x = self.embedding(x)
        x = self.linear_rate(x.transpose(1,2))
        x,hidden = self.r1(x.transpose(1,2))
        x = self.decoder(x.transpose(1,2))
        x = x.transpose(1,2)
        return x
    
g = data_gen(data_loader)
x,xi,y,yi = next(g)
model = Tnet(data_loader.pinyin_numbers,data_loader.char_numbers)
out= model(x)
out.shape

torch.Size([32, 50, 4461])

In [83]:
model = model.train().to(device=device,dtype=torch.float32)
optimizer = Adadelta(model.parameters(),lr=0.2)

In [84]:
bar = tqdm(data_gen(data_loader,size=128,loops=5000,device=device),total=5000)
for loop,(x,xi,y,yi) in enumerate(bar):
    
    optimizer.zero_grad()
    out=model(x)
    loss_ctc=F.ctc_loss(out.transpose(0,1),y,torch.full((x.shape[0],),x.shape[1],device=device),yi)
    loss_ctc.backward()
    optimizer.step()
    
#     _,ypred = torch.max(labels,dim=1,keepdim=False)
#     acc = ypred.eq(y.view_as(ypred)).sum().cpu().item()/(ypred.shape[0]*ypred.shape[1])
    if loop % 20 == 0:
        bar.set_postfix(ctc=f"{loss_ctc.item():0.6f}")

In [86]:
model=model.eval()

In [87]:
g=data_gen(data_loader,size=128,loops=5000,device=device)
x,xi,y,yi = next(g)
out = model(x)
out_array = out.detach().cpu().numpy()
out_array.shape

(128, 50, 4461)

In [89]:
[decoder],rate=tf.nn.ctc_greedy_decoder(out_array.transpose(1,0,2),
                        np.full((out_array.shape[0],),out_array.shape[1])
                        )

In [91]:
ss=[data_loader.array_to_char(s.numpy()) for s in y.cpu()]
ss=' '.join(ss)

In [92]:
ss_2=data_loader.array_to_char(decoder.values.numpy())

In [93]:
for s1,s2 in zip(ss.split('<pad>'),ss_2.split('<pad>')):
    print(s1)
    print(s2)



 特 斯 拉 的 后 续 中 端 产 品 的 投 放 
 特 斯 拉 的 后 续 终 端 产 品 的 投 放 
 是 确 保 证 券 执 法 有 效 性 的 重 要 条 件 
 是 确 保 证 券 执 法 有 效 性 的 重 要 条 件 
 整 合 国 际 国 内 的 一 流 资 源 
 整 合 国 际 国 内 的 一 流 资 源 
 各 种 产 品 细 节 的 打 造 受 到 业 内 外 追 捧 
 各 种 产 品 细 节 的 打 造 受 到 业 内 外 追 捧 
 而 谷 歌 将 蒙 受 人 才 流 失 带 来 的 损 失 
 而 谷 歌 将 蒙 售 人 才 流 失 带 来 的 损 失 
 三 里 屯 不 雅 视 频 网 传 手 机 号 码 存 疑 
 三 里 屯 不 雅 视 频 网 传 手 机 号 码 存 疑 
 目 前 还 在 安 阳 市 第 六 人 民 医 院 抢 救 
 目 前 还 在 安 阳 市 第 六 人 民 医 院 抢 救 
 分 别 索 赔 一 百 一 十 馀 万 元 
 分 别 索 赔 一 百 一 十 馀 万 元 
 甚 至 是 用 户 在 看 其 他 显 示 屏 例 如 电 视 机 
 甚 至 是 用 户 再 看 其 他 显 示 屏 例 如 电 视 机 
 导 致 生 成 巨 量 市 价 委 托 订 单 
 导 致 生 成 据 量 市 价 委 托 订 单 
 军 人 工 资 卡 辅 卡 大 量 优 惠 政 策 外 泄 保 密 性 超 强 
 军 人 工 资 卡 腐 卡 大 量 优 惠 政 策 外 保 密 性 超 强 
 大 家 都 以 为 跑 步 只 是 老 占 的 一 个 兴 趣 爱 好 
 大 家 都 以 为 跑 步 只 是 老 战 的 一 个 兴 趣 爱 好 
 鉴 于 此 经 研 究 决 定 海 峡 情 专 栏 原 定 三 月 底 结 束 现 延 至 五 月 三 十 一 日 特 此 告 知 
 鉴 于 此 经 研 究 决 定 海 峡 情 专 栏 原 定 三 月 底 结 束 现 延 至 五 月 三 十 一 日 特 此 告 知 
 且 一 直 以 来 也 没 有 和 女 儿 发 短 信 的 习 惯 
 且 一 直 以 来 也 没 有 和 女 儿 发 短 信 的 习 惯 
 千 亿 军 团 的 扩 容 在 今 年 或 许 能 够 实 现